<a href="https://colab.research.google.com/github/leomensah/LUNG-NODULE-DETECTION/blob/main/UNET_PLUS_WITH_AUGMENTATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install albumentations==0.4.6

     |████████████████████████████████| 117 kB 4.2 MB/s 
     |████████████████████████████████| 948 kB 28.3 MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-py3-none-any.whl size=65174 sha256=a8f467d519b458ee96520afea4b1039d6e58f2190d880b3f86cac1f1fd302e3d
  Stored in directory: /root/.cache/pip/wheels/cf/34/0f/cb2a5f93561a181a4bcc84847ad6aaceea8b5a3127469616cc
Successfully built albumentations
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Attempting uninstall: albumentations
    Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
import torch
from torch import nn

class VGGBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()
        self.relu = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_channels, middle_channels, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(middle_channels)
        self.conv2 = nn.Conv2d(middle_channels, out_channels, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        return out


class NestedUNet(nn.Module):
    def __init__(self, num_classes, input_channels=1,  **kwargs):
        super().__init__()

        nb_filter = [32, 64, 128, 256, 512]


        self.pool = nn.MaxPool2d(2, 2)
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = VGGBlock(input_channels, nb_filter[0], nb_filter[0])
        self.conv1_0 = VGGBlock(nb_filter[0], nb_filter[1], nb_filter[1])
        self.conv2_0 = VGGBlock(nb_filter[1], nb_filter[2], nb_filter[2])
        self.conv3_0 = VGGBlock(nb_filter[2], nb_filter[3], nb_filter[3])
        self.conv4_0 = VGGBlock(nb_filter[3], nb_filter[4], nb_filter[4])

        self.conv0_1 = VGGBlock(nb_filter[0]+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_1 = VGGBlock(nb_filter[1]+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_1 = VGGBlock(nb_filter[2]+nb_filter[3], nb_filter[2], nb_filter[2])
        self.conv3_1 = VGGBlock(nb_filter[3]+nb_filter[4], nb_filter[3], nb_filter[3])

        self.conv0_2 = VGGBlock(nb_filter[0]*2+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_2 = VGGBlock(nb_filter[1]*2+nb_filter[2], nb_filter[1], nb_filter[1])
        self.conv2_2 = VGGBlock(nb_filter[2]*2+nb_filter[3], nb_filter[2], nb_filter[2])

        self.conv0_3 = VGGBlock(nb_filter[0]*3+nb_filter[1], nb_filter[0], nb_filter[0])
        self.conv1_3 = VGGBlock(nb_filter[1]*3+nb_filter[2], nb_filter[1], nb_filter[1])

        self.conv0_4 = VGGBlock(nb_filter[0]*4+nb_filter[1], nb_filter[0], nb_filter[0])


        self.final = nn.Conv2d(nb_filter[0], num_classes, kernel_size=1)


    def forward(self, input):
        x0_0 = self.conv0_0(input)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.up(x1_3)], 1))

        output = self.final(x0_4)
        return output

In [ ]:
def iou_score(output, target):
    smooth = 1e-5

    if torch.is_tensor(output):
        output = torch.sigmoid(output).data.cpu().numpy()
    if torch.is_tensor(target):
        target = target.data.cpu().numpy()
    output_ = output > 0.5
    target_ = target > 0.5
    intersection = (output_ & target_).sum()
    union = (output_ | target_).sum()

    return (intersection + smooth) / (union + smooth)

def dice_coef(output, target):
    smooth = 1e-5

    # we need to use sigmoid because the output of Unet is logit.
    output = torch.sigmoid(output).view(-1).data.cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()
    

    return (2. * intersection + smooth) / (output.sum() + target.sum() + smooth)

def dice_coef2(output, target):
    "This metric is for validation purpose"
    smooth = 1e-5

    output = output.view(-1)
    output = (output>0.5).float().cpu().numpy()
    target = target.view(-1).data.cpu().numpy()
    intersection = (output * target).sum()
    

    return (2. * intersection + smooth) / (output.sum() + target.sum() + smooth)
def str2bool(v):
    if v.lower() in ['true', 1]:
        return True
    elif v.lower() in ['false', 0]:
        return False

def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
import torch
import torch.nn as nn
import torch.nn.functional as F

__all__ = ['BCEDiceLoss']

class BCEDiceLoss(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, input, target):
    bce = F.binary_cross_entropy_with_logits(input, target)
    smooth = 1e-5
    input = torch.sigmoid(input)
    num = target.size(0)
    input = input.view(num, -1)
    target = target.view(num, -1)
    intersection = (input * target)
    dice = (2. * intersection.sum(1) + smooth) / (input.sum(1) + target.sum(1) + smooth)
    dice = 1 - dice.sum() / num

    return 0.5 * bce + dice

In [ ]:
import os
import numpy as np
import glob
import torch
from torch.utils.data.dataset import Dataset
import torchvision.transforms.functional as TF
import torchvision
from torchvision import transforms
import albumentations as albu
from albumentations.pytorch import ToTensorV2


class MyLidcDataset(Dataset):
  def __init__(self, images_paths, mask_paths):
    self.image_paths = images_paths
    self.mask_paths = mask_paths

    self.albu_transformations =  albu.Compose([
            albu.ElasticTransform(alpha=1.1,alpha_affine=0.5,sigma=5,p=0.15),
            albu.HorizontalFlip(p=0.15),
            ToTensorV2()
        ])

    self.transformations = transforms.Compose([transforms.ToTensor()])

  def transform(self, image, mask):
    image = image.reshape(512,512,1)
    mask = mask.reshape(512,512,1)
    mask = mask.astype('uint8')
    augmented=  self.albu_transformations(image=image,mask=mask)
    image = augmented['image']
    mask = augmented['mask']
    mask= mask.reshape([1,512,512])
    image, mask = image.type(torch.FloatTensor), mask.type(torch.FloatTensor)
    return image, mask

  def __getitem__(self, index):
    image = np.load(self.image_paths[index])
    mask = np.load(self.mask_paths[index])
    image, mask = self.transform(image, mask)
    return image, mask

  def __len__(self):
    return len(self.image_paths)

In [ ]:
import pandas as pd
import os
from collections import OrderedDict
import yaml

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def train(train_loader, model, criterion, optimizer):
  avg_meters = {
      'loss': AverageMeter(),
      'iou': AverageMeter(),
      'dice': AverageMeter()
  }

  model.train()
  pbar = tqdm(total=len(train_loader))
  for input, target in train_loader:
    input = input.cuda()
    target = target.cuda()

    output = model(input)
    loss = criterion(output, target)
    iou = iou_score(output, target)
    dice = dice_coef(output, target)

    # compute gradient and do optimizing step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    avg_meters['loss'].update(loss.item(), input.size(0))
    avg_meters['iou'].update(iou, input.size(0))
    avg_meters['dice'].update(dice, input.size(0))

    postfix = OrderedDict([
            ('loss', avg_meters['loss'].avg),
            ('iou', avg_meters['iou'].avg),
            ('dice',avg_meters['dice'].avg)
        ])
    pbar.set_postfix(postfix)
    pbar.update(1)
  pbar.close()

  return OrderedDict([('loss', avg_meters['loss'].avg),
                      ('iou', avg_meters['iou'].avg),
                      ('dice',avg_meters['dice'].avg)])
    
def validate(val_loader, model, criterion):

  avg_meters = {'loss': AverageMeter(),
                'iou': AverageMeter(),
                'dice': AverageMeter()}

  # Swicth to evaluate mode
  model.eval()

  with torch.no_grad():
    pbar = tqdm(total=len(val_loader))
    for input, target in val_loader:
      input = input.cuda()
      target = target.cuda()

      output = model(input)
      loss = criterion(output, target)
      iou = iou_score(output, target)
      dice = dice_coef(output, target)

      avg_meters['loss'].update(loss.item(), input.size(0))
      avg_meters['iou'].update(iou, input.size(0))
      avg_meters['dice'].update(dice, input.size(0))

      postfix = OrderedDict([
                ('loss', avg_meters['loss'].avg),
                ('iou', avg_meters['iou'].avg),
                ('dice',avg_meters['dice'].avg)
            ])
      pbar.set_postfix(postfix)
      pbar.update(1)
    pbar.close()
  return OrderedDict([('loss', avg_meters['loss'].avg),
                        ('iou', avg_meters['iou'].avg),
                        ('dice',avg_meters['dice'].avg)])

In [ ]:
filename = 'UNET_PLUS_AUGMENTATION'

os.makedirs('/content/drive/MyDrive/data/output/model_outputs/{}'.format(filename), exist_ok=True)
print("Creating directory called", filename)

criterion = BCEDiceLoss().cuda()
cudnn.benchmark = True

# Creating the model
print("======= MODEL CREATING IN PROGRESS =====")
model = NestedUNet(num_classes=1)
model = model.cuda()

if torch.cuda.device_count() > 1:
  print("Let's use", torch.cuda.device_count(), "GPU")
  model = nn.DataParallel(model)
params = filter(lambda p: p.requires_grad, model.parameters())
epochs = 50
learning_rate = 1e-5
weight_decay = 1e-4
momentum = 0.9
nesterov = False
early_stopping = 50
optimizer = optim.Adam(params, lr=learning_rate, weight_decay=weight_decay)

# Directory of Image, Mask folder generated from the preprocessing stage ###
image_dir = '/content/drive/MyDrive/data/Nodule_data/image/'
mask_dir = '/content/drive/MyDrive/data/Nodule_data/mask/'
meta = pd.read_csv('/content/drive/MyDrive/data/Nodule_data/meta/meta.csv')

meta['original_image']= meta['original_image'].apply(lambda x:image_dir+ x +'.npy')
meta['mask_image'] = meta['mask_image'].apply(lambda x:mask_dir+ x +'.npy')

train_meta = meta[meta['data_split']=='Train']
val_meta = meta[meta['data_split']=='Validation']

# Get all *npy images into list for Train
train_image_paths = list(train_meta['original_image'])
train_mask_paths = list(train_meta['mask_image'])

# Get all *npy images into list for Validation
val_image_paths = list(val_meta['original_image'])
val_mask_paths = list(val_meta['mask_image'])

print("*"*50)
print("The length of image: {}, mask folders: {} for train".format(len(train_image_paths),len(train_mask_paths)))
print("The length of image: {}, mask folders: {} for validation".format(len(val_image_paths),len(val_mask_paths)))
print("Ratio between Val/ Train is {:2f}".format(len(val_image_paths)/len(train_image_paths)))
print("*"*50)

# Create Dataset
train_dataset = MyLidcDataset(train_image_paths, train_mask_paths)
val_dataset = MyLidcDataset(val_image_paths,val_mask_paths)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    num_workers=2)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=2,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=2)
log= pd.DataFrame(index=[],columns= ['epoch','lr','loss','iou','dice','val_loss','val_iou'])

best_dice = 0
trigger = 0

for epoch in range(epochs):
  # train for one epoch
  train_log = train(train_loader, model, criterion, optimizer)
  # evaluate on validation set
  val_log = validate(val_loader, model, criterion)

  print('Training epoch [{}/{}], Training BCE loss:{:.4f}, Training DICE:{:.4f}, Training IOU:{:.4f}, Validation BCE loss:{:.4f}, Validation Dice:{:.4f}, Validation IOU:{:.4f}'.format(
        epoch + 1, epochs, train_log['loss'], train_log['dice'], train_log['iou'], val_log['loss'], val_log['dice'],val_log['iou']))
  
  tmp = pd.Series([
      epoch,
      learning_rate,
      train_log['loss'],
      train_log['iou'],
      train_log['dice'],
      val_log['loss'],
      val_log['iou'],
      val_log['dice']
  ], index=['epoch', 'lr', 'loss', 'iou', 'dice', 'val_loss', 'val_iou','val_dice'])

  log = log.append(tmp, ignore_index=True)
  log.to_csv('/content/drive/MyDrive/data/output/model_outputs/{}/log.csv'.format(filename), index=False)

  trigger += 1

  if val_log['dice'] > best_dice:
    torch.save(model.state_dict(), '/content/drive/MyDrive/data/output/model_outputs/{}/model.pth'.format(filename))
    best_dice = val_log['dice']
    print("=> saved best model as validation DICE is greater than previous best DICE")
    trigger = 0

  torch.cuda.empty_cache()

Creating directory called UNET_PLUS_AUGMENTATION
======= MODEL CREATING IN PROGRESS =====
**************************************************
The length of image: 8349, mask folders: 8349 for train
The length of image: 2783, mask folders: 2783 for validation
Ratio between Val/ Train is 0.333333
**************************************************


100%|██████████| 1392/1392 [18:15<00:00,  1.27it/s, loss=1.12, iou=0.188, dice=0.00276]


Training epoch [1/50], Training BCE loss:1.1614, Training DICE:0.0018, Training IOU:0.0863, Validation BCE loss:1.1215, Validation Dice:0.0028, Validation IOU:0.1881
=> saved best model as validation DICE is greater than previous best DICE


100%|██████████| 1392/1392 [03:28<00:00,  6.68it/s, loss=1.07, iou=0.247, dice=0.00522]


Training epoch [2/50], Training BCE loss:1.0939, Training DICE:0.0043, Training IOU:0.2485, Validation BCE loss:1.0717, Validation Dice:0.0052, Validation IOU:0.2475
=> saved best model as validation DICE is greater than previous best DICE


...

100%|██████████| 1392/1392 [03:29<00:00,  6.64it/s, loss=0.412, iou=0.51, dice=0.627]


Training epoch [29/50], Training BCE loss:0.3181, Training DICE:0.7271, Training IOU:0.6115, Validation BCE loss:0.4121, Validation Dice:0.6275, Validation IOU:0.5105


100%|██████████| 1392/1392 [03:29<00:00,  6.64it/s, loss=0.425, iou=0.519, dice=0.629]


Training epoch [30/50], Training BCE loss:0.3155, Training DICE:0.7283, Training IOU:0.6128, Validation BCE loss:0.4253, Validation Dice:0.6286, Validation IOU:0.5185


 44%|████▍     | 1842/4174 [15:23<19:26,  2.00it/s, loss=0.305, iou=0.625, dice=0.741]

In [ ]:
import os
from glob import glob
from collections import OrderedDict
import numpy as np
import cv2
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import yaml
from sklearn.model_selection import train_test_split
from scipy import ndimage as ndi
from scipy.ndimage import label, generate_binary_structure
from sklearn.metrics.pairwise import euclidean_distances
from tqdm import tqdm
import pandas as pd

def save_output(output,output_directory,test_image_paths,counter):
  # This saves the predicted image into a directory. The naming convention will follow PI
  for i in range(output.shape[0]):
      label = test_image_paths[counter][-23:]
      label = label.replace('NI','PD')
      np.save(output_directory+'/'+label,output[i,:,:])
      #print("SAVED",output_directory+label+'.npy')
      counter+=1

  return counter

def calculate_fp(prediction_dir,mask_dir,distance_threshold=80):
  """This calculates the fp by comparing the predicted mask and orginal mask"""
  #TP,TN,FP,FN
  #FN will always be zero here as all the mask contains a nodule
  confusion_matrix =[0,0,0,0]
  # This binary structure enables the function to recognize diagnoally connected label as same nodule.
  s = generate_binary_structure(2,2)
  print('Length of prediction dir is ',len(os.listdir(prediction_dir)))
  for prediction in os.listdir(prediction_dir):
      #print(confusion_matrix)
      mask_id = prediction.replace('PD','MA')
      mask = np.load(mask_dir+'/'+mask_id)
      predict = np.load(prediction_dir+'/'+prediction)
      answer_com = np.array(ndi.center_of_mass(mask))
      # Patience is used to check if the patch has cropped the same image
      patience =0
      labeled_array, nf = label(predict, structure=s)
      if nf>0:
          for n in range(nf):
              lab=np.array(labeled_array)
              lab[lab!=(n+1)]=0
              lab[lab==(n+1)]=1
              predict_com=np.array(ndi.center_of_mass(labeled_array))
              if np.linalg.norm(predict_com-answer_com,2) < distance_threshold:
                  patience +=1
              else:
                  confusion_matrix[2]+=1
          if patience > 0:
              # Add to True Positive
              confusion_matrix[0]+=1
          else:
              # Add to False Negative
              # if the patience remains 0, and nf >0, it means that the slice contains both the TN and FP
              confusion_matrix[3]+=1

      else:
          # Add False Negative since the UNET didn't detect a cancer even when there was one
          confusion_matrix[3]+=1
  return np.array(confusion_matrix)

In [ ]:
NAME = 'UNET_PLUS_AUGMENTATION'
print('-'*20)

cudnn.benchmark = True

print("=> creating model {}".format(NAME))

model = NestedUNet(num_classes=1)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
print("Loading model file from {}".format(NAME))
model.load_state_dict(torch.load('/content/drive/MyDrive/data/output/model_outputs/{}/model.pth'.format(NAME)))
model = model.cuda()
IMAGE_DIR = '/content/drive/MyDrive/data/Nodule_data/image/'
MASK_DIR = '/content/drive/MyDrive/data/Nodule_data/mask/'

meta = pd.read_csv('/content/drive/MyDrive/data/Nodule_data/meta/meta.csv')
meta['original_image']= meta['original_image'].apply(lambda x:IMAGE_DIR+ x +'.npy')
meta['mask_image'] = meta['mask_image'].apply(lambda x:MASK_DIR+ x +'.npy')
test_meta = meta[meta['data_split']=='Test']

# Get all *npy images into list for Test(True Positive Set)
test_image_paths = list(test_meta['original_image'])
test_mask_paths = list(test_meta['mask_image'])

total_patients = len(test_meta.groupby('patient_id'))

print("*"*50)
print("The lentgh of image: {}, mask folders: {} for test".format(len(test_image_paths),len(test_mask_paths)))
print("Total patient number is :{}".format(total_patients))

OUTPUT_MASK_DIR = '/content/drive/MyDrive/data/Segmentation/{}'.format(NAME)
print("Saving OUTPUT files in directory {}".format(OUTPUT_MASK_DIR))
os.makedirs(OUTPUT_MASK_DIR,exist_ok=True)
test_dataset = MyLidcDataset(test_image_paths, test_mask_paths)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=2,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
    num_workers=2)

model.eval()
print(" ")
print("Printing the first 5 image directories...",test_image_paths[:5])
print("Printing the first 5 mask directories...",test_mask_paths[:5])

avg_meters = {'iou': AverageMeter(),
              'dice': AverageMeter()}
with torch.no_grad():
  counter = 0
  pbar = tqdm(total=len(test_loader))
  for input, target in test_loader:
    input = input.cuda()
    target = target.cuda()

    output = model(input)
    iou = iou_score(output, target)
    dice = dice_coef2(output, target)

    avg_meters['iou'].update(iou, input.size(0))
    avg_meters['dice'].update(dice, input.size(0))

    postfix = OrderedDict([
        ('iou', avg_meters['iou'].avg),
        ('dice',avg_meters['dice'].avg)
    ])

    output = torch.sigmoid(output)
    output = (output>0.5).float().cpu().numpy()
    output = np.squeeze(output,axis=1)
    #print(output.shape)

    counter = save_output(output,OUTPUT_MASK_DIR,test_image_paths,counter)
    pbar.set_postfix(postfix)
    pbar.update(1)
  pbar.close()
  print("="*50)

print('IoU: {:.4f}'.format(avg_meters['iou'].avg))
print('DICE:{:.4f}'.format(avg_meters['dice'].avg))

confusion_matrix = calculate_fp(OUTPUT_MASK_DIR ,MASK_DIR,distance_threshold=80)
print("="*50)
print("TP: {} FP:{}".format(confusion_matrix[0],confusion_matrix[2]))
print("FN: {} TN:{}".format(confusion_matrix[3],confusion_matrix[1]))
print("{:2f} FP/per Scan ".format(confusion_matrix[2]/total_patients))
print("="*50)
print(" ")
torch.cuda.empty_cache()

--------------------
=> creating model UNET_PLUS_AUGMENTATION
Loading model file from UNET_PLUS_AUGMENTATION
**************************************************
The lentgh of image: 2784, mask folders: 2784 for test
Total patient number is :717
Saving OUTPUT files in directory /content/drive/MyDrive/data/Segmentation/UNET_PLUS_AUGMENTATION
 
Printing the first 5 image directories... ['/content/drive/MyDrive/data/Nodule_data/image/0001_NI000_slice002.npy', '/content/drive/MyDrive/data/Nodule_data/image/0002_NI000_slice005.npy', '/content/drive/MyDrive/data/Nodule_data/image/0002_NI000_slice008.npy', '/content/drive/MyDrive/data/Nodule_data/image/0002_NI000_slice015.npy', '/content/drive/MyDrive/data/Nodule_data/image/0002_NI000_slice019.npy']
Printing the first 5 mask directories... ['/content/drive/MyDrive/data/Nodule_data/mask/0001_MA000_slice002.npy', '/content/drive/MyDrive/data/Nodule_data/mask/0002_MA000_slice005.npy', '/content/drive/MyDrive/data/Nodule_data/mask/0002_MA000_slice0

100%|██████████| 1392/1392 [36:28<00:00,  1.57s/it, iou=0.513, dice=0.627]


IoU: 0.5126
DICE:0.6268
Length of prediction dir is  2784
TP: 1786 FP:1374
FN: 998 TN:0
1.916318 FP/per Scan 
 
